# CSV 파일 매칭 검증
같은 카테고리의 product CSV와 review CSV를 비교하여 불일치 확인

In [6]:
import pandas as pd
import glob
import os
import re

data_dir = '/Users/yu_seok/Documents/workspace/nbCamp/Project/Why-pi/data'
product_files = glob.glob(os.path.join(data_dir, 'product_*.csv'))
review_files = glob.glob(os.path.join(data_dir, 'reviews_*.csv'))

print(f"product CSV: {len(product_files)}개")
print(f"reviews CSV: {len(review_files)}개\n")

def extract_category(filename):
    """product_메이크업_all_20260204.csv -> 메이크업_all_20260204"""
    basename = os.path.basename(filename)
    if basename.startswith('product_'):
        return basename.replace('product_', '').replace('.csv', '')
    elif basename.startswith('reviews_'):
        return basename.replace('reviews_', '').replace('.csv', '')
    return None

# 파일 매칭
product_dict = {extract_category(f): f for f in product_files}
review_dict = {extract_category(f): f for f in review_files}

print("="*80)
print("파일 매칭")
print("="*80)

all_categories = set(product_dict.keys()) | set(review_dict.keys())

for category in sorted(all_categories):
    has_product = category in product_dict
    has_review = category in review_dict
    
    status = "[OK]" if (has_product and has_review) else "[WARNING]"
    
    print(f"{status} {category}")
    if has_product:
        print(f"   product: {os.path.basename(product_dict[category])}")
    else:
        print(f"   product: 없음")
    
    if has_review:
        print(f"   reviews: {os.path.basename(review_dict[category])}")
    else:
        print(f"   reviews: 없음")
    print()

product CSV: 2개
reviews CSV: 2개

파일 매칭
[OK] 메이크업_20260204
   product: product_메이크업_20260204.csv
   reviews: reviews_메이크업_20260204.csv

[OK] 맨케어_all_20260204
   product: product_맨케어_all_20260204.csv
   reviews: reviews_맨케어_all_20260204.csv



In [7]:
matched_categories = set(product_dict.keys()) & set(review_dict.keys())

print("="*80)
print("카테고리별 product_code 매칭 검증")
print("="*80)
print()

total_only_product = 0
total_only_review = 0
total_matched = 0

for category in sorted(matched_categories):
    product_file = product_dict[category]
    review_file = review_dict[category]
    
    # CSV 읽기
    product_df = pd.read_csv(product_file)
    review_df = pd.read_csv(review_file)
    
    # product_code 추출
    product_codes = set(product_df['product_code'].dropna().astype(int))
    review_codes = set(review_df['product_code'].dropna().astype(int))
    
    # 매칭 확인
    both = product_codes & review_codes
    only_product = product_codes - review_codes
    only_review = review_codes - product_codes
    
    total_matched += len(both)
    total_only_product += len(only_product)
    total_only_review += len(only_review)
    
    print(f"{'='*80}")
    print(f"[{category}]")
    print(f"{'='*80}")
    print(f"  product CSV: {len(product_codes)}개")
    print(f"  reviews CSV: {len(review_codes)}개")
    print()
    print(f"  [OK] 매칭됨 (product & review 모두): {len(both)}개")
    
    if len(only_product) > 0:
        print(f"  [WARNING] product만 있음 (리뷰 없음): {len(only_product)}개")
        if len(only_product) <= 10:
            print(f"     코드: {sorted(only_product)}")
        else:
            print(f"     처음 10개: {sorted(only_product)[:10]}")
    
    if len(only_review) > 0:
        print(f"  [ERROR] review만 있음 (제품 정보 없음): {len(only_review)}개")
        if len(only_review) <= 10:
            print(f"     코드: {sorted(only_review)}")
        else:
            print(f"     처음 10개: {sorted(only_review)[:10]}")
    
    print()

카테고리별 product_code 매칭 검증

[메이크업_20260204]
  product CSV: 237개
  reviews CSV: 237개

  [OK] 매칭됨 (product & review 모두): 237개

[맨케어_all_20260204]
  product CSV: 73개
  reviews CSV: 72개

  [OK] 매칭됨 (product & review 모두): 72개
  [WARNING] product만 있음 (리뷰 없음): 1개
     코드: [1059931]



In [8]:
# 전체 요약
print(f"\n{'='*80}")
print("전체 요약")
print(f"{'='*80}")
print(f"  검증한 카테고리: {len(matched_categories)}개")
print(f"  [OK] 매칭됨: {total_matched}개")
print(f"  [WARNING] product만 있음: {total_only_product}개")
print(f"  [ERROR] review만 있음: {total_only_review}개")
print()

if total_only_product == 0 and total_only_review == 0:
    print("  [OK] 모든 파일이 완벽하게 매칭됩니다!")
elif total_only_review > 0:
    print(f"  [ERROR] {total_only_review}개 제품의 리뷰만 있고 제품 정보가 없습니다!")
elif total_only_product > 0:
    print(f"  [INFO] {total_only_product}개 제품은 리뷰가 없습니다 (정상)")


전체 요약
  검증한 카테고리: 2개
  [OK] 매칭됨: 309개
  [WARNING] product만 있음: 1개
  [ERROR] review만 있음: 0개

  [INFO] 1개 제품은 리뷰가 없습니다 (정상)


In [9]:
all_products = []

for product_file in product_files:
    df = pd.read_csv(product_file)
    if 'brand' in df.columns and 'name' in df.columns:
        all_products.append(df[['brand', 'name', 'product_code']])

# 전체 데이터 병합
if all_products:
    combined_df = pd.concat(all_products, ignore_index=True)
    
    print("="*80)
    print("전체 통계")
    print("="*80)
    print(f"총 제품 수: {len(combined_df)}개")
    print(f"고유 브랜드 수: {combined_df['brand'].nunique()}개")
    print()
    
    # 브랜드별 제품 수
    brand_counts = combined_df['brand'].value_counts()
    
    print("="*80)
    print("브랜드별 제품 수")
    print("="*80)
    for brand, count in brand_counts.items():
        if pd.notna(brand) and brand != '':
            print(f"{brand}: {count}개")
        else:
            print(f"[브랜드 없음]: {count}개")
    
    print()
    print("="*80)
    print("브랜드별 상품 목록")
    print("="*80)
    
    # 브랜드별로 그룹핑하여 상품명 출력
    for brand in sorted(combined_df['brand'].unique()):
        if pd.isna(brand) or brand == '':
            brand_name = "[브랜드 없음]"
        else:
            brand_name = brand
        
        products = combined_df[combined_df['brand'] == brand]
        
        print(f"\n[{brand_name}] ({len(products)}개)")
        print("-"*80)
        
        for idx, row in products.iterrows():
            print(f"  - {row['name']}")
else:
    print("제품 데이터가 없습니다.")

전체 통계
총 제품 수: 310개
고유 브랜드 수: 58개

브랜드별 제품 수
태그: 21개
입큰: 20개
트윙클팝: 20개
본셉 메이크업: 18개
프릴루드 딘토: 17개
투에딧: 17개
손앤박: 16개
드롭비 컬러즈: 15개
줌 바이 정샘물: 11개
코코가가: 10개
머지: 9개
클라뷰: 9개
밀크터치 디어씽: 8개
쉬크: 7개
플레이101 by 에뛰드: 7개
프렙 바이 비레디: 6개
데일리콤마: 6개
더랩 바이 블랑두: 6개
싸이닉: 6개
베리썸: 5개
도루코: 5개
파넬: 4개
코시에로: 4개
보닌: 4개
KAI: 4개
어퓨_더퓨어: 3개
[02: 3개
[01: 3개
코드글로컬러: 3개
오릭스: 3개
맨넨: 2개
6중날: 2개
닥터지오: 2개
도루코윈4: 2개
클리덤: 2개
도루코윈3: 2개
제이엠솔루션: 2개
멘넨: 2개
도루코윈: 2개
NEW: 2개
식물원: 2개
미팩토리: 2개
남성용다리털정리기: 1개
3중날: 1개
2중날: 1개
3중: 1개
도루코TG2플러스5P: 1개
[03: 1개
방수: 1개
전동: 1개
심플: 1개
비알티씨: 1개
캐릭터: 1개
스니키: 1개
[04: 1개
윙크걸: 1개
잇츠스킨: 1개
카이지루시: 1개

브랜드별 상품 목록

[2중날] (1개)
--------------------------------------------------------------------------------
  - 2중날 휴대용 면도기 10개입

[3중] (1개)
--------------------------------------------------------------------------------
  - 3중 날 시스템 면도기 리필 4개입

[3중날] (1개)
--------------------------------------------------------------------------------
  - 3중날 시스템 면도기 면도날 세트

[6중날] (2개)
-----------------------------------------